# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [13]:
import numpy as np
import pandas as pd
import random as rd

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [6]:
df_data = pd.read_csv('Data\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('Data\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

df_data.head(10)

,1,2,3,4,5,6,7,8
0,,,,,,,,
846160,0.388,0.329,0.690,0.900,0.626,0.621,0.399,0.370
820434,-0.296,-0.503,-0.454,-0.868,-0.721,-0.918,-0.486,-0.582
849103,-0.246,-0.935,-0.277,-0.175,-0.278,-0.075,-0.236,-0.417
846353,0.769,0.929,0.977,1.095,1.058,0.864,0.689,0.492
848613,-0.365,-0.760,-0.305,-0.293,-0.364,-0.155,-0.472,-0.606
846978,0.858,1.134,0.950,1.806,1.384,1.414,1.169,1.146
847332,-0.029,-0.018,-0.285,-0.813,-0.974,-0.936,-0.607,-0.514
847300,0.337,0.067,-0.762,-2.677,-2.772,-3.670,-1.356,-1.443
846924,0.552,0.743,0.581,1.027,0.754,0.596,0.712,0.506


## Cluster methods
### K-means (KMCA)

#### Vectorlengte berekenen
Een rij in het DataFrame is in feite een 8-dimensionale expressievector van een gen. Hiervan kan de Euclidische afstand gebruikt worden. Dit moet op veel plaatsen in het algoritme gebeuren, dus het is mooi om hier een losse functie van te maken.

In [299]:
def magnitude(vector):
    ''' 
     Preconditions: 'vector' is een pandas DataFrame
    Postconditions: returnt de lengte van de eerste rij van 'vector' als float
    '''
    return np.sqrt((vector**2).sum())

#### Data normaliseren
Elk gen heeft een 8-dimensionale expressievector, met daarin alle 8 expressiewaarden (r-waarden) van dat gen. Van deze vector wordt de Euclidische lengte berekend. Vervolgens worden van het gen alle r-waarden gedeeld door die lengte van de vector, waardoor je een eenheidsvector krijgt. De vector is nu genormaliseerd.

In [322]:
def normalize_data(data):
    ''' 
     Preconditions: 'data' is een pandas DataFrame
    Postconditions: returnt een pandas DataFrame waarvan elke rij genormaliseerd is tot eenheidsvector
    '''
    normalized = data.copy()
    indexlist = normalized.index.tolist()
    for row in indexlist:
        vector_magnitude = magnitude(normalized.loc[row]) # vector_magnitude moet bepaald worden vóórdat je de waarden in de rij gaat vervangen
        for col in data.columns:
            normalized.loc[row,col] = normalized.loc[row,col] / vector_magnitude
    return normalized

#### Clustercentrum berekenen
Een cluster bestaat uit een verzameling vectoren. Het clustercentrum is het gemiddelde van alle vectoren in een cluster.

In [8]:
def calculate_center(vectors):
    '''
     Precondition: 'vectors' is een pandas DataFrame
    Postcondition: returnt een pandas DataFrame, die het centrum van de vectoren die in 'vectors' zijn meegegeven aangeeft
    '''
    center = vectors.sum()/len(vectors.index)
    return center

#### K-means clusteralgoritme

In [317]:
def KMCA(data, k, seed=0):
    '''
     Precondition: 'data' is een pandas DataFrame;
                   'k' is een integer dat aangeeft hoeveel clusters gemaakt moeten worden;
                   'k' > 0;
                   'k' <= het aantal rijen in 'data'
    Postcondition: returnt een pandas DataFrame met bij elke index van 'data' een kolom 'cluster' waarin het bijbehorend cluster
                   van die rij staat
    '''
    assert k > 0 and k <= len(data.index)
    clustered = data.copy()
    E_list = []
    centers = {}
    
    # INITIAL CLUSTERING
    rd.seed(seed)
    for i in range(k):
        random = rd.choice(clustered.index.tolist())
        centers[i]=clustered.loc[random]
        
    # REASSIGNMENT
    PLACEHOLDER = 0
    while PLACEHOLDER < 3:
        for row in clustered.index.tolist():
            clustered.loc[row,'cluster_nr']='None'
            vector = clustered.loc[row].drop('cluster_nr')
            for i in centers:
                if clustered.loc[row,'cluster_nr']=='None' or magnitude(centers[i]-vector) < magnitude(centers[clustered.loc[row]['cluster_nr']]-vector):
                    clustered.loc[row,'cluster_nr']=i
        for i in centers:
            centers[i]=calculate_center(clustered[clustered['cluster_nr']==i])
        PLACEHOLDER += 1
        
    # RETURN
    return clustered,E_list

In [321]:
seed=0
clustered_data,E_progression = KMCA(normalize_data(df_data)[:50],5,seed)
clustered_data.head(50)

,1,2,3,4,5,6,7,8,cluster_nr
0,,,,,,,,,
846160,0.239776,0.203315,0.426405,0.556181,0.386855,0.383765,0.246573,0.228652,4
820434,-0.164504,-0.279546,-0.252314,-0.482397,-0.400701,-0.510185,-0.270098,-0.323451,0
849103,-0.211364,-0.803356,-0.237999,-0.150361,-0.238859,-0.064440,-0.202772,-0.358288,3
846353,0.309072,0.373379,0.392670,0.440096,0.425225,0.347254,0.276919,0.197742,4
848613,-0.285344,-0.594142,-0.238438,-0.229057,-0.284563,-0.121174,-0.368993,-0.473750,3
846978,0.239997,0.317198,0.265731,0.505168,0.387127,0.395519,0.326988,0.320555,4
847332,-0.016210,-0.010061,-0.159301,-0.454426,-0.544417,-0.523176,-0.339282,-0.287300,0
847300,0.058719,0.011674,-0.132770,-0.466438,-0.482991,-0.639457,-0.236268,-0.251427,0
846924,0.278244,0.374520,0.292861,0.517674,0.380065,0.300422,0.358894,0.255057,4


# Vanaf hier is niet meer relevant, heb alleen geen tijd gehad om het weg te halen
### Own cluster method:
#### Quantile assignment
De namen van de gebruikte kwantielen worden gedefinieerd in de lijst 'quantiles'. De 'pandas.qcut' functie wordt uitgevoerd op elk van de r-dimensies (kolommen van df_data). Deze functie neemt van een kolom de waarden van alle genen (rijen) en berekent aan de hand daarvan in welk kwantiel elke waarde hoort. De hoeveelheid kwantielen die worden gebruikt hangt af van de lengte van de lijst 'quantiles', zodat er meer kwantielen worden gevormd wanneer andere namen worden toegevoegd aan de lijst.

De rijen code die gevolgd worden door #test zijn niet belangrijk voor het programma, maar geven een inzicht in de berekening van kwantielen voor alle r-dimensies.

In [92]:
def assign_quantiles(data,quantiles):
    '''
    Preconditions : 'data' is een pandas DataFrame; 'quantiles' is een lijst van strings
    Postconditions: returnt een pandas DataFrame, met als waarden de kwantielen waarin elke waarde uit het 'data' DataFrame behoort
    '''
    leveled = data.copy()
    for rX in range(1,len(leveled.columns)+1):
        print('r'+str(rX)) #test
        for q_nr in range(len(quantiles)): #test
            print('\t', quantiles[q_nr], leveled[rX].quantile(q=(q_nr+1)*(1/len(quantiles)))) #test
        leveled[rX] = pd.qcut(leveled[rX], len(quantiles), labels=quantiles)
    return leveled

quantiles = ['1st','2nd','3rd']
assign_quantiles(df_data,quantiles).head(10)

r1
	 1st -0.10400000000000023
	 2nd 0.4959999999999999
	 3rd 2.041
r2
	 1st -0.2550000000000004
	 2nd 0.5429999999999994
	 3rd 3.835
r3
	 1st -0.3950000000000002
	 2nd 0.5179999999999998
	 3rd 3.051
r4
	 1st -0.5470000000000002
	 2nd 0.809
	 3rd 4.5569999999999995
r5
	 1st -0.3450000000000001
	 2nd 0.7070000000000001
	 3rd 3.315
r6
	 1st -0.29300000000000004
	 2nd 0.4709999999999999
	 3rd 2.535
r7
	 1st -0.218
	 2nd 0.5439999999999997
	 3rd 2.611
r8
	 1st -0.254
	 2nd 0.31199999999999967
	 3rd 1.544


,1,2,3,4,5,6,7,8
0,,,,,,,,
846160,2nd,2nd,3rd,3rd,2nd,3rd,2nd,3rd
820434,1st,1st,1st,1st,1st,1st,1st,1st
849103,1st,1st,2nd,2nd,2nd,2nd,1st,1st
846353,3rd,3rd,3rd,3rd,3rd,3rd,3rd,3rd
848613,1st,1st,2nd,2nd,1st,2nd,1st,1st
846978,3rd,3rd,3rd,3rd,3rd,3rd,3rd,3rd
847332,2nd,2nd,2nd,1st,1st,1st,1st,1st
847300,2nd,2nd,1st,1st,1st,1st,1st,1st
846924,3rd,3rd,3rd,3rd,3rd,3rd,3rd,3rd


### Data normalization

In [171]:
def normalize_data(data):
    normalized = data.copy()
    indexlist = normalized.index.tolist()
    for row in indexlist:
        normalized.loc[row,'vector_length'] = np.sqrt(sum(normalized.loc[846160,:]**2))#np.sqrt(sum(normalized.loc[row,:]**2))
    return normalized

normalize_data(df_data)

,1,2,3,4,5,6,7,8,vector_length
0,,,,,,,,,
846160,0.388,0.329,0.690,0.900,0.626,0.621,0.399,0.370,1.618179
820434,-0.296,-0.503,-0.454,-0.868,-0.721,-0.918,-0.486,-0.582,2.288451
849103,-0.246,-0.935,-0.277,-0.175,-0.278,-0.075,-0.236,-0.417,2.288451
846353,0.769,0.929,0.977,1.095,1.058,0.864,0.689,0.492,2.288451
848613,-0.365,-0.760,-0.305,-0.293,-0.364,-0.155,-0.472,-0.606,2.288451
...,...,...,...,...,...,...,...,...,...
760857,1.243,1.824,1.569,2.671,1.644,1.305,1.541,1.094,2.288451
805218,0.810,0.641,0.545,0.973,1.137,0.518,0.711,0.355,2.288451
945014,-0.486,-0.743,-0.066,0.046,0.364,0.176,0.172,0.006,2.288451


## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [5]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
0,
846160,0
820434,4
849103,2
846353,3
848613,2


### save results:

In [6]:
df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')